In [2]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

ImportError: cannot import name 'base_models'

In [ ]:
# very few datapoints just to demonstrate how to use models

# preprocessing
from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()
dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 5)
dataset_s , frequency_list_s = text_preprocessing.keep_labels(preprocessded_dataset,
                                                           keep_ratio=0.1)

# text preprocessing
slice_dataset = text_preprocessing.dataset_slice(dataset_s,ratio=0.01)

#tf-idf
all_sentences, all_labels = text_preprocessing.tf_idf(slice_dataset)

# split the dataset
X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sentences, all_labels)

In [ ]:
from multilab.models import BinaryRe
Bm = BinaryRe(X_train, y_train, X_test,y_test)
print(Bm.train())

# four base models

# BinaryRelevance
# ClassifierChain
# LabelPowerset
# Mlknn


In [ ]:
print(Bm.BinaryRe())
print(Bm.classfier_chain())
print(Bm.powerset())
print(Bm.mlknn())